In [ ]:
import numpy as np
import tensorflow as tf
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 12})

In [ ]:
# Load sample image data
sam_img = np.load('Val_data.npz')['val_X'] # Sample validation data as an exmaple.
sam_img_num = sam_img.shape[0]
num_sam = 10 # Sample validation data has 10 sample images per category

In [ ]:
# Define functions
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1,1,3))
def img_proc(sam_img):
    imgs_proc = np.zeros(sam_img.shape)
    for idx, img in enumerate(sam_img):
        img = np.asarray(img).astype('float32')
        img = img - vgg_mean
        imgs_proc[idx] = img[:,:,::-1]
    return imgs_proc

def get_layer_rep(li,imgs):
    n_imgs = imgs.shape[0]
    op = graph.get_tensor_by_name(relus[li]+':0')
    ft_sh = (n_imgs, op.shape[1], op.shape[2], op.shape[3])
    rep_ = np.zeros((ft_sh),dtype='float32')
    for i,img in enumerate(imgs):
        img = np.expand_dims(img,0)
        rp = sess.run(op, {x:img, is_training:False})
        rep_[i] = rp
    rep_ = rep_.reshape(n_imgs,-1)
    return rep_ # 16,1000 or 16

In [ ]:
# Load model
sess = tf.Session()
saver = tf.train.import_meta_graph('net-55.ckpt.meta')
saver.restore(sess, 'net-55.ckpt')
graph = tf.get_default_graph()
# Load placeholder
x = graph.get_tensor_by_name("Placeholder:0") 
is_training = graph.get_tensor_by_name("Placeholder_2:0")
# Get relu layer
relus = [op.name for op in graph.get_operations() if op.type=='Relu']
relus = relus[:14] + ['fc8/Conv2D']

# CNN RDM

In [ ]:
depth = 15
cnnrdms = np.zeros((depth,16,16))
sam_img_proc = img_proc(sam_img) # Image preprocessing 
for li in range(depth):
    rep_ = get_layer_rep(li,sam_img_proc)
    # PCA
    std_ = StandardScaler()
    x_scld = std_.fit_transform(rep_)
    pca = PCA(n_components=0.90)
    pca.fit(x_scld)
    x_ = pca.transform(x_scld)
    # Average to 16x16
    x_16 = np.zeros((16,x_.shape[1]))
    for ci in range(16):
        x_16[ci] = np.mean(x_[ci*num_sam:(ci+1)*num_sam],axis=0)
    cnnrdms[li] = 1-np.corrcoef(x_16.transpose(), rowvar=False)

# Visualize CNN RDM

In [ ]:
# Mask 
mask = np.zeros((16,16))
mask[np.triu_indices_from(mask)] = True
# Title list
tlist = []
for li in range(1,14):
    tlist.append('Conv '+str(li))
tlist += ['FC','Output']
# Tickslabels
labels = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight',
          'Nine','Bed','Bird','Cat','Dog','House','Tree']
# Default font size
plt.rcParams.update({'font.size': 20})

In [ ]:
# Plot RDM
# Autorange
f,axes = plt.subplots(3,5, figsize=(38,20))
ax = axes.flat
for li in range(15):
    ax[li].set_title(tlist[li],fontsize=40)
    sns.heatmap(cnnrdms[li],mask=mask,cmap='jet',ax=ax[li],xticklabels=labels,yticklabels=labels,square=True,cbar=True) #,vmin=0,vmax=2
    plt.tight_layout()
plt.show()
plt.close()

# Min 0 Max 2
f,axes = plt.subplots(3,5, figsize=(38,20))
ax = axes.flat
for li in range(15):
    ax[li].set_title(tlist[li],fontsize=40)
    sns.heatmap(cnnrdms[li],mask=mask,cmap='jet',ax=ax[li],xticklabels=labels,yticklabels=labels,square=True,vmin=0,vmax=2)
    plt.tight_layout()
plt.show()
plt.close()

# RSA between neural RDM and CNN RDM

In [ ]:
iu = np.triu_indices(16,1)
def rsa(neural_rdm,cnnrdm):
    rho, p = stats.spearmanr(neural_rdm[iu],cnnrdm[iu])
    return rho, p 

In [ ]:
# Load sample neural RDM made by '03_MVPA_Neural_RDMs.ipynb'
l = np.load('Sample_Neural_RDM.npz',allow_pickle=True)
brain_rdm = l['rdm']
brain_inform_list = l['info_list']

# Number of searchlight voxels
nvv = brain_rdm.shape[0]

# RSA
rsa_rslt = np.zeros((depth,64,76,64))
rsa_rslt_pval = np.zeros((depth,64,76,64))
for li in range(depth):
    vol_rho = np.zeros((64,76,64))
    vol_pval = np.zeros((64,76,64))
    for vi in range(nvv):
        idx = brain_inform_list[vi][1]
        neural_rdm = brain_rdm[vi]
        vol_rho[idx], vol_pval[idx] = rsa(neural_rdm, cnnrdms[li])
    rsa_rslt[li], rsa_rslt_pval[li] = vol_rho, vol_pval

# Layer assignment map
- In this tutorial, you would use rho values rather than t-statistic from group data.

In [ ]:
p_threshold = 0.05
rsa_rslt[(rsa_rslt_pval > p_threshold)] = 0 # uncorrected p-values used in this sample code
# Mask 
tmpsum = np.sum((rsa_rslt>0),axis=0)
tmpsum2 = np.sum((rsa_rslt<0),axis=0)
msk_idx = np.where((tmpsum > 0)&(tmpsum2!=15))
# Maximally similar layer
max_info = np.argmax(rsa_rslt.transpose(1,2,3,0)[msk_idx],axis=1) # vec,depth
max_info += 1
# Assign
assign_map = np.zeros((64,76,64))
assign_map[msk_idx] = max_info

In [ ]:
# If you want to save assigment map obtained from sample data, you can run this cell.
# Please set the path to save.
savepath = '/path/to/save/assigned_map.nii'

import nibabel as nib

affine = np.array([[   3. ,   -0. ,   -0. ,  -94.5],
       [  -0. ,    3. ,   -0. , -130.5],
       [   0. ,    0. ,    3. ,  -76.5],
       [   0. ,    0. ,    0. ,    1. ]])

def save_nifti(vol, savepath, zero_to_nan=True):
    if zero_to_nan:
        vol[(vol==0)] = np.nan
    img1 = nib.Nifti1Image(vol,affine=affine)
    nib.save(img1,savepath)
    
save_nifti(assign_map, savepath)

In [ ]:
# Save CNN RDMs for running the code '06-1_Cosine_CNN_Code'
np.savez('Sample_CNN_RDM.npz',cnnrdms=cnnrdms)